### 필요한 패키지 import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
import seaborn as sns
import math
# 사이킷런
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
# 시계열
from fbprophet import Prophet
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
# 경고 숨기기
import warnings
warnings.filterwarnings(action='ignore')

Importing plotly failed. Interactive plots will not work.


### MAE(Mean Absolute Error)

In [53]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

mae_scorer = make_scorer(mae)
mae_scorer

make_scorer(mae)

### RMSLE
#### 과대평가 된 항목보다는 과소평가 된 항목에 페널티를 준다.

#### 오차(Error)를 제곱(Square)해서 평균(Mean)한 값의 제곱근(Root)으로 값이 작을수록 정밀도가 높다.

#### 0에 가까운 값이 나올수록 정밀도가 높은 값이다.

#### Submissions are evaluated one the Root Mean Squared Logarithmic Error (RMSLE)

In [75]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)

    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)

    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)

    # 평균을 낸다.
    mean_difference = difference.mean()

    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)

    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

### 데이터 준비

In [3]:
df=pd.read_csv('data/jeju_data_ver1/201901-202003.csv')
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)

In [103]:
df_202004=pd.read_csv('data/jeju_data_ver1/202004.csv')

In [43]:
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


### 4월부터 7월까지 예측 템플릿 만들기

In [4]:
#4월부터 7월까지
# 예측 템플릿 만들기
year = 202000
months  = [4,5,6,7]
CARD_SIDO_NMs=sorted(df['CARD_SIDO_NM'].unique())
STD_CLSS_NMs=sorted(df['STD_CLSS_NM'].unique())
temp = []
for month in months:
    for i in CARD_SIDO_NMs:
        for j in STD_CLSS_NMs:
            temp.append([year+month,i,j,0])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT'])

In [ ]:
temp.index.name = 'id'
temp.to_csv('submission_4to7.csv', encoding='utf-8-sig')

In [39]:
df_set=df.loc[(df['CARD_SIDO_NM']=='강원')&(df['STD_CLSS_NM']=='건강보조식품 소매업')]
df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
df_re=df_re.reset_index()
df_re['REG_YYMM']=df_re['REG_YYMM'].apply(lambda x:str(x))
df_re['REG_YYMM']=df_re['REG_YYMM'].apply(lambda x:x[:4]+'-'+x[4:])
df_re['REG_YYMM']=pd.to_datetime(df_re['REG_YYMM'])
df_re.set_index('REG_YYMM', inplace=True)
df_re.head()

,CARD_SIDO_NM,STD_CLSS_NM,AMT
REG_YYMM,,,
2019-01-01,강원,건강보조식품 소매업,148354746
2019-02-01,강원,건강보조식품 소매업,146715871
2019-03-01,강원,건강보조식품 소매업,109439025
2019-04-01,강원,건강보조식품 소매업,106548473
2019-05-01,강원,건강보조식품 소매업,148115738


In [40]:
df_re= df_re.drop(['CARD_SIDO_NM','STD_CLSS_NM'], axis=1)

### 분해법 이용한 다음에 불규칙 성분에 arima나 평활 적용하면 어때?

In [41]:
result_mul = seasonal_decompose(df_re, model='multiplicative')
result.plot()
plt.show()

ValueError: x must have 2 complete cycles requires 24 observations. x only has 15 observation(s)

In [ ]:
result_add = seasonal_decompose(df, model='additive')

### Exponential Smoothing (지수평활법 - Simple, Holt's, Holt-Winter's)
#### 1) Simple Exponential Smoothing
- trend나 seasonality 반영을 하지 못함
- level 정도만 수평선으로 나옴
$$ F_t = F_{t-1} + \alpha(D_{t-1} - F_{t-1}) $$
$$ F_t = (1 - \alpha)F_{t-1} + \alpha D_{t-1} $$
 
- Ft  : 현재 시점의 예측 값
- Ft−1  : 이전 시점의 예측 값
- Dt−1  : 이전 시점의 실제 값
- α  : smoothing 요소, 0 <  α  < 1

![](...%0Aa8UWlm5jdQAAAABJRU5ErkJggg==)

In [61]:
for i in df['CARD_SIDO_NM'].unique():
    for j in df['STD_CLSS_NM'].unique():
        df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
        df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
        try:
            ses_model=SimpleExpSmoothing(df_re['AMT'])
            ses_result = ses_model.fit()
            ses_pred = ses_result.forecast(4)
            temp.loc[(temp['REG_YYMM']=='202004')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=ses_pred.iloc[0]
            temp.loc[(temp['REG_YYMM']=='202005')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=ses_pred.iloc[1]
            temp.loc[(temp['REG_YYMM']=='202006')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=ses_pred.iloc[2]
            temp.loc[(temp['REG_YYMM']=='202007')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=ses_pred.iloc[3]
        except:
            temp.loc[(temp['REG_YYMM']=='202004')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202005')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202006')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202007')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15

C:\anaconda\envs\jiwoo\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiw

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

...

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been 

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index w

### 4월 데이터로 평가 진행

In [ ]:
rmsle(predicted_values, actual_values)

In [62]:
temp.index.name = 'id'
temp.to_csv('submission_ses.csv', encoding='utf-8-sig')
temp.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.19622e+08
1,202004,강원,골프장 운영업,2.9158e+09
2,202004,강원,과실 및 채소 소매업,1.12032e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.06568e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,11800


### 2)  Holt's Exponential Smoothing
- trend로 데이터를 예측하기 위해 Simple Exponential Smoothing에서 확장한 것이다.
- 예측을 위한 식 외에 level smoothing을 위한 식과 trend smoothing을 위한 식이 포함된다
- 생성된 예측은 선형적으로 나타나기 때문에 예측 범위가 멀어질 수록 over-forecast 되는 경향이 있다.
- Forecast equation :
$$ \hat{y}_{t+h|t} = l_t + hb_t $$
 
- Level equation :

$$ l_{t} = \alpha y_t + (1-\alpha)(l_{t-1} + b_{t-1}) $$
 
- Trend Equation :

$$ b_t = \beta^*(l_t - l_{t-1}) + (1 - \beta^*)b_{t-1} $$
 
- lt  : t 시점에서의 level(수준)의 추정을 나타낸다.

- bt  : t 시점에서의 추세(경사)의 추정을 나타낸다.
- α  : level(수준)에 대한 smoothing parameter이고 다음과 같음 범위를 갖는다. 0 <  α  < 1
- β∗  : trend(추세)에 대한 smoothing parameter이고 다음과 같은 범위를 갖는다. 0 <  β∗  < 1
- level에 대한 식  lt 는 t 시간에 대한 관측치  yt 와 훈련 예측  lt−1+bt−1 의 가중 평균을 나타낸다.
- trend에 대한 식  bt 는  (lt−lt−1) 에 근거한 t시간에 대한 추정치와 이전 추정치인  bt−1 의 가중 평균을 나타낸다.

![](...%0AAElFTkSuQmCC)

In [44]:
for i in df['CARD_SIDO_NM'].unique():
    for j in df['STD_CLSS_NM'].unique():
        df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
        df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
        try:
            holt_model = Holt(np.array(df_re['AMT']))
            holt_result = holt_model.fit()
            holt_pred = holt_result.forecast(4)
            temp.loc[(temp['REG_YYMM']=='202004')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=holt_pred.iloc[0]
            temp.loc[(temp['REG_YYMM']=='202005')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=holt_pred.iloc[1]
            temp.loc[(temp['REG_YYMM']=='202006')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=holt_pred.iloc[2]
            temp.loc[(temp['REG_YYMM']=='202007')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=holt_pred.iloc[3]
        except:
            temp.loc[(temp['REG_YYMM']=='202004')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202005')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202006')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202007')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15

### 4월 데이터로 평가 진행

In [ ]:
rmsle(predicted_values, actual_values)

In [ ]:
temp.index.name = 'id'
temp.to_csv('submission_holt.csv', encoding='utf-8-sig')
temp.head()

### 3) Holt-Winter's Exponential Smoothing
- Holt-Winter's 방법은 seasonality를 반영하기 위해 Holt's 방법에서 확장된 것이다.
- Holt-Winter's 방법은 예측식과 3개의 smoothing 식으로 구성되어 있다.
- level에 대한 식  l_t   
- trend에 대한 식  b_t
- seasonal에 대한 요소  s_t
- smoothing parameter에 해당하는  α ,  β∗ ,  γ 
- seasonality의 빈도를 나타내기 위한  m 
- seasonal이 변화하는 형태에 따라 두 가지 방법이 있다.
- additive : seasonal의 변화가 일정하게 지속될 때
- multiplicative : seasonal의 변화가 level에 비례적일 때
- Forecast equation
$$ \hat{y}_{t+h|t} = l_t + hb_t + s_{t + h - m(k+1)} $$
 
- Level equation
$$ l_t = \alpha(y_t - s_{t-m}) + (1 - \alpha)(l_{t-1} + b_{t-1}) $$
 
- Trend equation
$$ b_t = \beta^*(l_t - l_{t-1}) + (1 - \beta^*)b_{t-1} $$
 
- Seasonal equation
$$ s_t = \gamma(y_t - l_{t-1} - b_{t-1}) + (1 - \gamma)s_{t-m} $$

 
- level에 대한 식은 t 시간에 대한 조정된 seasonally 관측치  (y_t−s_t−m) 와 non-seasonally 예측치  (lt−1+bt−1) 의 가중 평균을 나타낸다.
- trend에 대한 식은 Holt's 선형식에서와 동일하다.
- seasonal에 대한 식은 현재 seasonal 지수  (yt−lt−1−bt−1) 와 이전 seasonal 지수( m 기간 전) 사이의 가중 평균을 나타낸다.

In [171]:
for i in df['CARD_SIDO_NM'].unique():
    for j in df['STD_CLSS_NM'].unique():
        df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
        df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
        size = int(len(df_re) * 0.8)
        train = df_re.iloc[:size]
        test = df_re.iloc[size:]
        # 최적값을 찾기 위한 기준
        es_score = 1000000000
        best_period = 0
        # 각 category 최적의 seasonal period를 찾기 위한 탐색
        for p in range(2,(len(train)//2) + 1):
            try:
                es_model = ExponentialSmoothing(np.array(train['AMT']),seasonal_periods=p, trend='add', seasonal='add')
                es_result = es_model.fit()
                es_pred = es_result.forecast(len(test))
                if es_score > mae(test['AMT'], es_pred):
                    es_score = mae(test['AMT'], es_pred)
                    best_period = p
            except:
                continue
        # 최적의 parameter을 이용한 4개월 예측
        try:
            set_es_model = ExponentialSmoothing(np.array(df_re['AMT']), seasonal_periods=best_period, trend='add', seasonal='add')
            set_es_result = set_es_model.fit()
            set_es_pred = set_es_result.forecast(4)
            temp.loc[(temp['REG_YYMM']=='202004')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=set_es_pred.iloc[0]
            temp.loc[(temp['REG_YYMM']=='202005')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=set_es_pred.iloc[1]
            temp.loc[(temp['REG_YYMM']=='202006')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=set_es_pred.iloc[2]
            temp.loc[(temp['REG_YYMM']=='202007')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=set_es_pred.iloc[3]
        except:
            temp.loc[(temp['REG_YYMM']=='202004')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202005')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202006')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            temp.loc[(temp['REG_YYMM']=='202007')&(temp['CARD_SIDO_NM']==i)&(temp['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15

KeyboardInterrupt: 

In [60]:
temp.index.name = 'id'
temp.to_csv('submission_Holt-Winter1.csv', encoding='utf-8-sig')

In [172]:
temp.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,1.21388e+08
1,202004,강원,골프장 운영업,3.41939e+09
2,202004,강원,과실 및 채소 소매업,1.1117e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.04808e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,11800


### AMOUNT랑 관련성이 높게 나온 CNT랑 CSTMR_CNT를 시계열로 예측하고 예측된 CNT와 CSTMR_CNT를 가지고 AMOUNT를 회귀분석해서 예측값을 뽑아보자!

In [48]:
df.corr(method='pearson')

,REG_YYMM,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
REG_YYMM,1.000000,-0.009399,-0.005525,0.001432,0.003167,0.002144
SEX_CTGO_CD,-0.009399,1.000000,-0.023134,0.008258,-0.005424,0.000469
FLC,-0.005525,-0.023134,1.000000,0.006774,0.041099,-0.001011
CSTMR_CNT,0.001432,0.008258,0.006774,1.000000,0.866339,0.930356
AMT,0.003167,-0.005424,0.041099,0.866339,1.000000,0.778102
CNT,0.002144,0.000469,-0.001011,0.930356,0.778102,1.000000


#### CNT와 CSTMR_CNT가 상관성이 높아서 둘중에 한 변수만 넣는 게 좋겠다는 판단
#### AMT와 상관성이 더 높은 CSTMR_CNT를 넣는 것이 좋아보임.

### 1) Simple Exponential Smoothing

In [95]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
for i in df['CARD_SIDO_NM'].unique():
    for j in df['STD_CLSS_NM'].unique():
        df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
        df_amt=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
        df_cstmr=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['CSTMR_CNT'].agg({'CSTMR_CNT': np.sum})
        try:
            ses_model=SimpleExpSmoothing(np.array(df_cstmr['CSTMR_CNT']))
            ses_result = ses_model.fit()
            ses_pred = ses_result.forecast(4)
            
            x_train=df_cstmr['CSTMR_CNT']
            y_train=df_amt['AMT']
            LinearR=LinearRegression()
            LinearR.fit(x_train.values.reshape(-1,1),y_train) 
            prediction=LinearR.predict(ses_pred.reshape(-1,1))
            
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=prediction[0]
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=prediction[3]
            
        except:
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_amt['AMT'])/15
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_amt['AMT'])/15

In [96]:
submission.index.name = 'id'
submission.to_csv('submission_cstmr_ses.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.213883e+08
1,202004,강원,골프장 운영업,1.968374e+09
2,202004,강원,과실 및 채소 소매업,1.111698e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,2.138632e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.180000e+04


### 2)  Holt's Exponential Smoothing

In [63]:
df=pd.read_csv('data/jeju_data_ver1/201901-202003.csv')

In [136]:
CARD_SIDO_NMs=df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs=df['STD_CLSS_NM'].unique()
AMT04=[]
AMT07=[]
df_re=df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum}).reset_index()
for i in CARD_SIDO_NMs:
    for j in STD_CLSS_NMs:
        df_set=df_re.loc[(df_re['CARD_SIDO_NM']==i)&(df_re['STD_CLSS_NM']==j)]
        if len(df_set)==15:
            ses_model=SimpleExpSmoothing(df_set['AMT'])
            ses_result = ses_model.fit()
            ses_pred = ses_result.forecast(4)
            AMT04.append([i,j,ses_pred.iloc[0]])
            AMT07.append([i,j,ses_pred.iloc[3]])
        else:
            if (df_set['REG_YYMM']==201904).any():
                AMT04.append([i,j,int(df_set.loc[df_set['REG_YYMM']==201904,'AMT'])])
            else:
                AMT04.append([i,j,np.sum(df_set['AMT'])/len(df_set)])
            if (df_set['REG_YYMM']==201907).any():
                AMT07.append([i,j,int(df_set.loc[df_set['REG_YYMM']==201907,'AMT'])])
            else:
                AMT07.append([i,j,np.sum(df_set['AMT'])/len(df_set)])

In [137]:
AMT04=pd.DataFrame(AMT04,columns=['CARD_SIDO_NM','STD_CLSS_NM','AMT'])
AMT07=pd.DataFrame(AMT07,columns=['CARD_SIDO_NM','STD_CLSS_NM','AMT'])
AMT04=AMT04.fillna(0)
AMT07=AMT07.fillna(0)

In [87]:
ans=pd.read_csv('data/jeju_data_ver1/202004.csv')
ans_group=ans.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].sum().reset_index(name='AMT') 
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission=submission[submission['REG_YYMM']==202004]
ans_group=ans_group[ans_group['REG_YYMM']==202004]
ans = submission.merge(ans_group, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
ans.fillna(0,inplace=True)

In [145]:
ans['AMT']

0      8.882399e+07
1      4.708347e+09
2      1.121029e+09
3      1.436078e+07
4      0.000000e+00
           ...     
692    1.373636e+09
693    1.891104e+10
694    1.412150e+07
695    4.505074e+08
696    9.328420e+06
Name: AMT, Length: 697, dtype: float64

In [144]:
AMT04['AMT']

0      1.196217e+08
1      2.915798e+09
2      1.120324e+09
3      3.065677e+07
4      2.486183e+06
           ...     
692    9.933600e+05
693    9.360360e+05
694    6.770500e+06
695    0.000000e+00
696    0.000000e+00
Name: AMT, Length: 697, dtype: float64

In [142]:
rmsle(ans['AMT'],AMT04['AMT'])

9.125858158122817

In [4]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
CARD_SIDO_NMs=df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs=df['STD_CLSS_NM'].unique()
AMT04=[]
AMT07=[]
df_amt_original=df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum}).reset_index()
df_cstmr_original=df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['CSTMR_CNT'].agg({'CSTMR_CNT': np.sum}).reset_index()
for i in CARD_SIDO_NMs:
    for j in STD_CLSS_NMs:
        df_amt = df_amt_original.loc[(df_amt_original['CARD_SIDO_NM']==i)&(df_amt_original['STD_CLSS_NM']==j), :]
        df_cstmr = df_cstmr_original.loc[(df_cstmr_original['CARD_SIDO_NM']==i)&(df_cstmr_original['STD_CLSS_NM']==j), :]
        try:
            holt_model = Holt(np.array(df_cstmr['CSTMR_CNT']))
            holt_result = holt_model.fit()
            holt_pred = holt_result.forecast(4)
            
            x_train=df_cstmr['CSTMR_CNT']
            y_train=df_amt['AMT']
            
            LinearR=LinearRegression()
            LinearR.fit(x_train.values.reshape(-1,1),y_train) 
            prediction=LinearR.predict(holt_pred.reshape(-1,1))
            
            AMT04.append([i,j,prediction[0]])
            AMT07.append([i,j,prediction[3]])
            
        except:
            
            AMT04.append([i,j,np.sum(df_amt['AMT'])/15])
            AMT07.append([i,j,np.sum(df_amt['AMT'])/15])

In [5]:
AMT04=pd.DataFrame(AMT04,columns=['CARD_SIDO_NM','STD_CLSS_NM','AMT'])
AMT07=pd.DataFrame(AMT07,columns=['CARD_SIDO_NM','STD_CLSS_NM','AMT'])

In [10]:
AMT04['REG_YYMM'] = 202004
AMT07['REG_YYMM'] = 202007
df4_7=pd.concat([AMT04,AMT07],axis=0)
predictions=df4_7[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [12]:
predictions.loc[predictions['AMT']<0,'AMT']=0
predictions.to_csv('final_cstmr_holt.csv', encoding='utf-8-sig')
predictions.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,1.005137e+08
1,202004,강원,골프장 운영업,1.968370e+09
2,202004,강원,과실 및 채소 소매업,9.562036e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,8.778789e+06
4,202004,강원,그외 기타 스포츠시설 운영업,7.731352e+05


### 3) Holt-Winter's Exponential Smoothing

In [15]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
CARD_SIDO_NMs=df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs=df['STD_CLSS_NM'].unique()
AMT04=[]
AMT07=[]

df_amt_original = df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].sum().reset_index()
df_cstmr_original = df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['CSTMR_CNT'].sum().reset_index()

In [16]:
for i in CARD_SIDO_NMs:
    print(i)
    for j in STD_CLSS_NMs:
        print('\t', j)
        df_amt = df_amt_original.loc[(df_amt_original['CARD_SIDO_NM']==i)&(df_amt_original['STD_CLSS_NM']==j), :]
        df_cstmr = df_cstmr_original.loc[(df_cstmr_original['CARD_SIDO_NM']==i)&(df_cstmr_original['STD_CLSS_NM']==j), :]
        size = int(len(df_cstmr) * 0.8)
        train = df_cstmr.iloc[:size]
        test  = df_cstmr.iloc[size:]
        # 최적값을 찾기 위한 기준
        es_score = 1000000000
        best_period = 0
        # 각 category 최적의 seasonal period를 찾기 위한 탐색
        for p in range(2,(len(train)//2) + 1):
            try:
                es_model = ExponentialSmoothing(np.array(train['CSTMR_CNT']),seasonal_periods=p, trend='add', seasonal='add')
                es_result = es_model.fit()
                es_pred = es_result.forecast(len(test))
                if es_score > mae(test['AMT'], es_pred):
                    es_score = mae(test['AMT'], es_pred)
                    best_period = p
            except:
                continue
        # 최적의 parameter을 이용한 4개월 예측
        try:
            set_es_model = ExponentialSmoothing(np.array(df_cstmr['CSTMR_CNT']), seasonal_periods=best_period, trend='add', seasonal='add')
            set_es_result = set_es_model.fit()
            set_es_pred = set_es_result.forecast(4)

            x_train=df_cstmr['CSTMR_CNT']
            y_train=df_amt['AMT']
            LinearR=LinearRegression()
            LinearR.fit(x_train.values.reshape(-1,1),y_train) 
            prediction=LinearR.predict(set_es_pred.reshape(-1,1))
            AMT04.append([i,j,prediction[0]])
            AMT07.append([i,j,prediction[3]])
        except:
            AMT04.append([i,j,np.sum(df_amt['AMT'])/15])
            AMT07.append([i,j,np.sum(df_amt['AMT'])/15])

강원
	 건강보조식품 소매업
	 골프장 운영업
	 과실 및 채소 소매업
	 관광 민예품 및 선물용품 소매업
	 그외 기타 스포츠시설 운영업
	 그외 기타 종합 소매업
	 기타 대형 종합 소매업
	 기타 외국식 음식점업
	 기타 주점업
	 기타음식료품위주종합소매업
	 마사지업
	 비알콜 음료점업
	 빵 및 과자류 소매업
	 서양식 음식점업
	 수산물 소매업
	 슈퍼마켓
	 스포츠 및 레크레이션 용품 임대업
	 여관업
	 욕탕업
	 육류 소매업
	 일반유흥 주점업
	 일식 음식점업
	 전시 및 행사 대행업
	 중식 음식점업
	 차량용 가스 충전업
	 차량용 주유소 운영업
	 체인화 편의점
	 피자 햄버거 샌드위치 및 유사 음식점업
	 한식 음식점업
	 호텔업
	 화장품 및 방향제 소매업
	 휴양콘도 운영업
	 여행사업
	 자동차 임대업
	 면세점
	 버스 운송업
	 택시 운송업
	 기타 수상오락 서비스업
	 내항 여객 운송업
	 그외 기타 분류안된 오락관련 서비스업
	 정기 항공 운송업
경기
	 건강보조식품 소매업
	 골프장 운영업
	 과실 및 채소 소매업
	 관광 민예품 및 선물용품 소매업
	 그외 기타 스포츠시설 운영업
	 그외 기타 종합 소매업
	 기타 대형 종합 소매업
	 기타 외국식 음식점업
	 기타 주점업
	 기타음식료품위주종합소매업
	 마사지업
	 비알콜 음료점업
	 빵 및 과자류 소매업
	 서양식 음식점업
	 수산물 소매업
	 슈퍼마켓
	 스포츠 및 레크레이션 용품 임대업
	 여관업
	 욕탕업
	 육류 소매업
	 일반유흥 주점업
	 일식 음식점업
	 전시 및 행사 대행업
	 중식 음식점업
	 차량용 가스 충전업
	 차량용 주유소 운영업
	 체인화 편의점
	 피자 햄버거 샌드위치 및 유사 음식점업
	 한식 음식점업
	 호텔업
	 화장품 및 방향제 소매업
	 휴양콘도 운영업
	 여행사업
	 자동차 임대업
	 면세점
	 버스 운송업
	 택시 운송업
	 기타 수상오락 서비스업
	 내항 여객 운송업
	 그외 기타 분류안된 오락관련 서비스업
	 정기 항공 운송

	 서양식 음식점업
	 수산물 소매업
	 슈퍼마켓
	 스포츠 및 레크레이션 용품 임대업
	 여관업
	 욕탕업
	 육류 소매업
	 일반유흥 주점업
	 일식 음식점업
	 전시 및 행사 대행업
	 중식 음식점업
	 차량용 가스 충전업
	 차량용 주유소 운영업
	 체인화 편의점
	 피자 햄버거 샌드위치 및 유사 음식점업
	 한식 음식점업
	 호텔업
	 화장품 및 방향제 소매업
	 휴양콘도 운영업
	 여행사업
	 자동차 임대업
	 면세점
	 버스 운송업
	 택시 운송업
	 기타 수상오락 서비스업
	 내항 여객 운송업
	 그외 기타 분류안된 오락관련 서비스업
	 정기 항공 운송업


In [20]:
AMT04_df = pd.DataFrame(AMT04, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'])
AMT07_df = pd.DataFrame(AMT07, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'])

AMT04_df['REG_YYMM'] = 202004
AMT07_df['REG_YYMM'] = 202007

In [21]:
predictions = pd.concat([AMT04_df, AMT07_df], axis=0)
predictions = predictions[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

In [22]:
predictions.tail()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
692,202007,충북,택시 운송업,4.409407e+05
693,202007,충북,기타 수상오락 서비스업,2.484974e+06
694,202007,충북,내항 여객 운송업,4.123600e+06
695,202007,충북,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
696,202007,충북,정기 항공 운송업,0.000000e+00


In [23]:
predictions.index.name = 'id'
predictions.loc[predictions['AMT']<0,'AMT']=0
predictions.to_csv('final_cstmr_holt&winters.csv',encoding='utf-8-sig')
predictions.tail()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
692,202007,충북,택시 운송업,4.409407e+05
693,202007,충북,기타 수상오락 서비스업,2.484974e+06
694,202007,충북,내항 여객 운송업,4.123600e+06
695,202007,충북,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
696,202007,충북,정기 항공 운송업,0.000000e+00


#### 4월 데이터 추가

In [27]:
ans=pd.read_csv('data/jeju_data_ver1/202004.csv')

In [30]:
ans=pd.read_csv('data/jeju_data_ver1/202004.csv')
CARD_SIDO_NMs=ans['CARD_SIDO_NM'].unique()
STD_CLSS_NMs=ans['STD_CLSS_NM'].unique()
ans_group=ans.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].sum().reset_index(name='AMT')                                                                           

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,88823988
1,202004,강원,골프장 운영업,4708346820
2,202004,강원,과실 및 채소 소매업,1121028924
3,202004,강원,관광 민예품 및 선물용품 소매업,14360780
4,202004,강원,그외 기타 스포츠시설 운영업,227200
...,...,...,...,...
605,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
606,202004,충북,한식 음식점업,18911036160
607,202004,충북,호텔업,14121500
608,202004,충북,화장품 및 방향제 소매업,450507431


In [53]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
ans_group=ans_group[ans_group['REG_YYMM']==202004]
ans = submission.merge(ans_group, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
ans.fillna(0,inplace=True)

In [54]:
predictions=predictions[predictions['REG_YYMM']==202004]

### CV<0.1보다 작으면 로그변환해서 지수평활, 이동평균 예측

In [109]:
CARD_SIDO_NMs=df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs=df['STD_CLSS_NM'].unique()
df_amt_original = df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum}).reset_index()
pred_list=[]

### 단순지수평활

In [55]:
CARD_SIDO_NMs=df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs=df['STD_CLSS_NM'].unique()
df_amt_original = df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum}).reset_index()
pred_list=[]
pred_list=[]
CV=[]
for i in CARD_SIDO_NMs:
    for j in STD_CLSS_NMs:
        try:
            df_amt = df_amt_original.loc[(df_amt_original['CARD_SIDO_NM']==i)&(df_amt_original['STD_CLSS_NM']==j),['REG_YYMM', 'AMT']]
            cv=df_amt['AMT'].std()/df_amt['AMT'].mean()
            CV.append([202004,i,j,cv])
        except:
            cv=0
        if (cv<=0.13):
            df_amt['log_AMT'] = np.log(df_amt['AMT'])
            ses_model=SimpleExpSmoothing(df_amt['log_AMT'])
            ses_result = ses_model.fit()
            ses_pred = ses_result.forecast(4)
            pred_4 = np.exp(ses_pred.iloc[0])
            pred_list.append([202004,i, j, pred_4])
            pred_7=np.exp(ses_pred.iloc[3])
            pred_list.append([202007,i,j,pred_7])
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
pred_log=pd.DataFrame(pred_list, columns=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'])
pred_log=pred_log.sort_values(by='REG_YYMM',ascending=True)
pred_log.to_csv('final_prediction.csv',encoding='utf-8-sig')

In [56]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)

In [57]:
pred_log=pd.DataFrame(pred_list, columns=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'])
pred_log=pred_log.sort_values(by='REG_YYMM',ascending=True)
pred_log.to_csv('final_prediction.csv',encoding='utf-8-sig')

In [157]:
pred_list=[]
for i in CARD_SIDO_NMs:
    for j in STD_CLSS_NMs:
        try:
            df_amt = df_amt_original.loc[(df_amt_original['CARD_SIDO_NM']==i)&(df_amt_original['STD_CLSS_NM']==j),['REG_YYMM', 'AMT']]
            cv=df_amt['AMT'].std()/df_amt['AMT'].mean()
        except:
            cv=0
        if (cv<=0.1):
            ses_model=SimpleExpSmoothing(df_amt['AMT'])
            ses_result = ses_model.fit()
            ses_pred = ses_result.forecast(4)
            pred=ses_pred.iloc[0]
            pred_list.append([202004,i, j,pred])           

In [158]:
pred_nonlog=pd.DataFrame(pred_list, columns=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'])
total_nonlog = pred_nonlog.merge(ans_group, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')

In [159]:
print(rmsle(total_log['AMT_x'],total_log['AMT_y']))
print(rmsle(total_nonlog['AMT_x'],total_nonlog['AMT_y']))

0.0973530421161592
0.091647937395331


In [ ]:
names

In [163]:
df_re=df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])[['AMT']].sum().reset_index()

In [166]:
df_re.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500


In [167]:
CARD_SIDO = df['CARD_SIDO_NM'].unique()
STD_CLSS = df['STD_CLSS_NM'].unique()
names=[]
for i in CARD_SIDO:
    for j in STD_CLSS:
        df_set=df_re.loc[(df_re['CARD_SIDO_NM']==i)&(df_re['STD_CLSS_NM']==j)]
        if len(df_set)!=15:
            if len(df_set)!=0:
                for [reg_yymm, sido, clss, amt] in df_set.values:
                    names.append([reg_yymm, sido, clss, amt])

In [169]:
pd.DataFrame(names, columns=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'])

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,면세점,3501010
1,201904,강원,면세점,113730
2,201907,강원,면세점,431090
3,202001,강원,면세점,625010
4,202002,강원,면세점,147940
...,...,...,...,...
478,201909,충북,내항 여객 운송업,3151000
479,201910,충북,내항 여객 운송업,12018500
480,201911,충북,내항 여객 운송업,7056000
481,201912,충북,내항 여객 운송업,262000


### ETS(Error Trend Season)

In [ ]:
from statsmodels.tsa.seasonal import STL
stl = STL(co2, seasonal=13)

### Seasonal-Trend decomposition using LOESS (STL)

In [ ]:
ets_stl = ExponentialSmoothing((time_log) ,seasonal_periods=12 ,trend='add', seasonal='add').fit()
ets_stl1 = ets_stl.forecast(len(time_log))